# Player Trueskill

### Helpful Resources
[Trueskill Documentation](https://trueskill.org/#version-0-4-4)  
[Trueskill for DOTA2 Players - Kaggle](https://www.kaggle.com/devinanzelmo/dota-2-skill-rating-with-trueskill)  
[Collection of Trueskill Examples](https://python.hotexamples.com/examples/trueskill/TrueSkill/-/python-trueskill-class-examples.html)  

In [1]:
# Housekeeping
import requests
import datetime
import pandas as pd
import numpy as np
import io
import os
import trueskill
import itertools
import math

import oeutils

pd.options.display.max_rows = 100
pd.options.display.max_columns = 500

workingdir = '../data/'
years = ['2018', '2019', '2020', '2021']

#oeutils.download(workingdir, years)
df = oeutils.read('../data/', years)
df = oeutils.clean(df, split_on='player', keep_identities=False, keep_leagues=False, keep_columns=False)

/home/paulperrone/source/lol-analysis/notebooks/oeutils.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oe_data['gameid'] = oe_data['gameid'].str.strip()


In [2]:
df.shape

(188865, 117)

In [3]:
df['league'].unique()

array(['LPL', 'LCK', 'CK', 'VCS', 'LCS.A', 'EU LCS', 'LLN', 'OPL', 'LMS',
       'TCL', 'CBLOL', 'CLS', 'NA LCS', 'TCS', 'LJL', 'LCL', 'CIS CL',
       'BRCC', 'GPL', 'EM', 'MSI', 'OCS', 'WCS', 'UL', 'NASG', 'DC',
       'KeSPA', 'LEC', 'LLA', 'TRA', 'BIG', 'LFL', 'LCS', 'UKLC', 'LDL',
       'UPL', 'PCS', 'Riot', 'MSC', 'RCL', 'CU', 'NLC', 'NEST', 'BL',
       'PGN', 'LCK CL', 'GLL', 'SL', 'PRM', 'CBLOL.A', 'EBL', 'BM'],
      dtype=object)

In [4]:
df.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
348,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,1,Blue,top,Duke,Invictus Gaming,Ornn,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,3,12,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9631,363.4340,0.150020,315.4340,NaN,13,0.4906,6,0.2264,2,NaN,NaN,10791,7272,274.4151,0.188550,8870.0,NaN,218.0,216.0,2,1,0,8.2264,3292.0,4758.0,90.0,3287.0,4640.0,88.0,5.0,118.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5280.0,7775.0,150.0,5060.0,7477.0,144.0,220.0,298.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
349,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,2,Blue,jng,Ning,Invictus Gaming,Kha'Zix,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,0,5,12,4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6924,261.2830,0.107854,693.8113,NaN,48,1.8113,24,0.9057,10,NaN,NaN,10831,7312,275.9245,0.189587,9775.0,NaN,194.0,36.0,158,96,8,7.3208,3170.0,3745.0,67.0,2988.0,3386.0,58.0,182.0,359.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4660.0,5588.0,100.0,4290.0,4957.0,80.0,370.0,631.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
350,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,3,Blue,mid,Rookie,Invictus Gaming,Orianna,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,8,12,4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19985,754.1509,0.311303,274.6415,NaN,10,0.3774,7,0.2642,5,NaN,NaN,12165,8646,326.2642,0.224176,9310.0,NaN,266.0,246.0,20,16,4,10.0377,3633.0,5236.0,108.0,3378.0,5108.0,92.0,255.0,128.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,5389.0,7541.0,157.0,5211.0,8079.0,150.0,178.0,-538.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
351,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,4,Blue,bot,JackeyLove,Invictus Gaming,Ezreal,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,7,12,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23623,891.4340,0.367971,321.2075,NaN,14,0.5283,6,0.2264,2,NaN,NaN,14028,10509,396.5660,0.272482,13075.0,NaN,276.0,257.0,19,8,11,10.4151,3992.0,3286.0,97.0,3438.0,3018.0,88.0,554.0,268.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,6800.0,5569.0,159.0,5138.0,5263.0,134.0,1662.0,306.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
352,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,5,Blue,sup,Baolan,Invictus Gaming,Tahm Kench,A

In [5]:
lcs = df[df['league'].isin(['LCS', 'NA LCS'])]
lcs.shape

(7590, 117)

In [6]:
lcs['player'].unique()

array(['Impact', 'Xmithie', 'Pobelter', 'Doublelift', 'Olleh', 'Hauntzer',
       'MikeYeung', 'Bjergsen', 'Zven', 'Mithy', 'Ssumday', 'Meteos',
       'Ryu', 'Cody Sun', 'aphromoo', 'zig', 'Akaadian', 'PowerOfEvil',
       'Arrow', 'LemonNation', 'Solo', 'Lira', 'FEBIVEN', 'Apollo',
       'Hakuho', 'Lourlo', 'Contractz', 'Hai', 'Deftly', 'Matt', 'Huni',
       'Dardoch', 'Fenix', 'Altec', 'Adrian', 'Flame', 'AnDa', 'Keane',
       'WildTurtle', 'Stunt', 'ZionSpartan', 'Reignover', 'huhi',
       'Stixxay', 'Biofrost', 'Licorice', 'Svenskeren', 'Jensen',
       'Sneaky', 'Smoothie', 'Shrimp', 'Fly', 'JayJ', 'Dhokla', 'Damonte',
       'PapaChau', 'Grig', 'Goldenglue', 'Keith', 'Zeyzal', 'Santorin',
       'Kwon', 'Mickey', 'Big', 'Allorim', 'Feng', 'Gate', 'Blaber',
       'Lost', 'Moon', 'Piglet', 'Vulcan', 'bobqin', 'FallenBandit',
       'Wiggily', 'Rikara', 'CoreJJ', 'Nisqy', 'Bang', 'Broken Blade',
       'Crown', 'Neo', 'V1per', 'Froggen', 'Rush', 'Panda', 'Auto',
       'Soligo

In [7]:
lcs = lcs.sort_values(by=['date', 'player'])

In [8]:
player_map = {
    
}

def replace_player_name(player_to_replace):
    if player_to_replace in player_map.keys():
        return player_map[player_to_replace]
    else:
        return player_to_replace

lcs['player'].unique()

array(['Bjergsen', 'Doublelift', 'Hauntzer', 'Impact', 'MikeYeung',
       'Mithy', 'Olleh', 'Pobelter', 'Xmithie', 'Zven', 'Akaadian',
       'Arrow', 'Cody Sun', 'LemonNation', 'Meteos', 'PowerOfEvil', 'Ryu',
       'Ssumday', 'aphromoo', 'zig', 'Apollo', 'Contractz', 'Deftly',
       'FEBIVEN', 'Hai', 'Hakuho', 'Lira', 'Lourlo', 'Matt', 'Solo',
       'Adrian', 'Altec', 'AnDa', 'Dardoch', 'Fenix', 'Flame', 'Huni',
       'Keane', 'Stunt', 'WildTurtle', 'Biofrost', 'Jensen', 'Licorice',
       'Reignover', 'Smoothie', 'Sneaky', 'Stixxay', 'Svenskeren',
       'ZionSpartan', 'huhi', 'Shrimp', 'Fly', 'JayJ', 'Dhokla',
       'Damonte', 'PapaChau', 'Grig', 'Goldenglue', 'Keith', 'Zeyzal',
       'Kwon', 'Santorin', 'Big', 'Mickey', 'Allorim', 'Feng', 'Gate',
       'Blaber', 'Lost', 'Moon', 'Piglet', 'Vulcan', 'bobqin',
       'FallenBandit', 'Wiggily', 'Rikara', 'CoreJJ', 'Nisqy', 'Bang',
       'Broken Blade', 'Crown', 'Neo', 'Froggen', 'V1per', 'Rush',
       'Panda', 'Auto', 'Soligo

In [9]:
lcs.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
1699,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,8,Red,mid,Bjergsen,TSM,Ryze,Kog'Maw,Tahm Kench,Ezreal,Braum,Shen,1687,0,1,0,0,1,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0356,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,8313,295.6609,0.301534,325.6787,406.0225,13,0.4624,5,0.1778,2,24.0,0.8536,10993,7276,258.7789,0.255883,10750.0,NaN,301.0,293.0,8,8,0,10.7054,3620.0,5005.0,103.0,3818.0,4866.0,95.0,-198.0,139.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,5540.0,8006.0,162.0,5897.0,7708.0,146.0,-357.0,298.0,16.0,0.0,0.0,0.0,2.0,0.0,0.0
1695,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,4,Blue,bot,Doublelift,Team Liquid,Tristana,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,5,0,5,11,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,11534,410.2193,0.251691,168.4410,361.2448,14,0.4979,11,0.3912,3,40.0,1.4226,14046,10329,367.3622,0.253797,12375.0,NaN,289.0,253.0,36,13,8,10.2786,3510.0,3360.0,94.0,3563.0,3173.0,97.0,-53.0,187.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,6213.0,5507.0,142.0,5395.0,5280.0,155.0,818.0,227.0,-13.0,2.0,1.0,0.0,0.0,0.0,1.0
1697,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,6,Red,top,Hauntzer,TSM,Gnar,Kog'Maw,Tahm Kench,Ezreal,Braum,Shen,1687,0,0,3,0,1,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0356,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,7391,262.8690,0.268091,576.0640,422.3829,15,0.5335,1,0.0356,3,26.0,0.9247,10105,6388,227.1962,0.224653,9075.0,NaN,271.0,271.0,0,0,0,9.6384,3119.0,4254.0,78.0,3650.0,4637.0,77.0,-531.0,-383.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,5087.0,7378.0,140.0,5824.0,7885.0,130.0,-737.0,-507.0,10.0,0.0,0.0,1.0,0.0,3.0,0.0
1692,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,1,Blue,top,Impact,Team Liquid,Gangplank,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,3,0,6,11,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,16858,599.5732,0.367870,386.8880,319.9881,14,0.4979,2,0.0711,2,24.0,0.8536,13474,9757,347.0184,0.239742,12133.0,NaN,254.0,248.0,6,1,4,9.0338,3650.0,4637.0,77.0,3119.0,4254.0,78.0,531.0,383.0,-1.0,0.0,1.0,0.0,0.0,0.0,1.0,5824.0,7885.0,130.0,5087.0,7378.0,140.0,737.0,507.0,-10.0,0.0,3.0,0.0,0.0,0.0,1.0
1698,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,20

In [10]:
input_data = lcs[['gameid', 'date', 'player', 'side', 'team', 'position', 'result']]
input_data.head()

,gameid,date,player,side,team,position,result
1699,TRLH1/1002440062,2018-01-20 20:50:09,Bjergsen,Red,TSM,mid,0
1695,TRLH1/1002440062,2018-01-20 20:50:09,Doublelift,Blue,Team Liquid,bot,1
1697,TRLH1/1002440062,2018-01-20 20:50:09,Hauntzer,Red,TSM,top,0
1692,TRLH1/1002440062,2018-01-20 20:50:09,Impact,Blue,Team Liquid,top,1
1698,TRLH1/1002440062,2018-01-20 20:50:09,MikeYeung,Red,TSM,jng,0


In [11]:
columns = ['gameid', 'date', \
           'blue_team', \
           'blue_top_name', \
           'blue_jng_name', \
           'blue_mid_name', \
           'blue_bot_name', \
           'blue_sup_name',\
           'blue_team_result', \
           'red_team', \
           'red_top_name', \
           'red_jng_name', \
           'red_mid_name', \
           'red_bot_name', \
           'red_sup_name']

lcs_rating = pd.DataFrame(columns=columns)

## Trueskill

In [12]:
ts = trueskill.TrueSkill(draw_probability=0.0)

In [13]:
player_ratings_dict = dict()
for i in input_data['player'].unique():
    player_ratings_dict[i] = ts.create_rating()
    
print(player_ratings_dict)

{'Bjergsen': trueskill.Rating(mu=25.000, sigma=8.333), 'Doublelift': trueskill.Rating(mu=25.000, sigma=8.333), 'Hauntzer': trueskill.Rating(mu=25.000, sigma=8.333), 'Impact': trueskill.Rating(mu=25.000, sigma=8.333), 'MikeYeung': trueskill.Rating(mu=25.000, sigma=8.333), 'Mithy': trueskill.Rating(mu=25.000, sigma=8.333), 'Olleh': trueskill.Rating(mu=25.000, sigma=8.333), 'Pobelter': trueskill.Rating(mu=25.000, sigma=8.333), 'Xmithie': trueskill.Rating(mu=25.000, sigma=8.333), 'Zven': trueskill.Rating(mu=25.000, sigma=8.333), 'Akaadian': trueskill.Rating(mu=25.000, sigma=8.333), 'Arrow': trueskill.Rating(mu=25.000, sigma=8.333), 'Cody Sun': trueskill.Rating(mu=25.000, sigma=8.333), 'LemonNation': trueskill.Rating(mu=25.000, sigma=8.333), 'Meteos': trueskill.Rating(mu=25.000, sigma=8.333), 'PowerOfEvil': trueskill.Rating(mu=25.000, sigma=8.333), 'Ryu': trueskill.Rating(mu=25.000, sigma=8.333), 'Ssumday': trueskill.Rating(mu=25.000, sigma=8.333), 'aphromoo': trueskill.Rating(mu=25.000, si

In [14]:
### Lambda Apply w Position as Key
def setup_match(df, gameid, position):
    #blue players
    blue_frame = df.query(f'(gameid == "{gameid}") and (side == "Blue")')
    blue_top_name = blue_frame.query(f'(gameid == "{gameid}") and (side == "Blue") and (position == "top")')
    blue_top_name = blue_top_name['player'].iloc[0]
    blue_jng_name = blue_frame.query(f'(gameid == "{gameid}") and (side == "Blue") and (position == "jng")')
    blue_jng_name = blue_jng_name['player'].iloc[0]
    blue_mid_name = blue_frame.query(f'(gameid == "{gameid}") and (side == "Blue") and (position == "mid")')
    blue_mid_name = blue_mid_name['player'].iloc[0]
    blue_bot_name = blue_frame.query(f'(gameid == "{gameid}") and (side == "Blue") and (position == "bot")')
    blue_bot_name = blue_bot_name['player'].iloc[0]
    blue_sup_name = blue_frame.query(f'(gameid == "{gameid}") and (side == "Blue") and (position == "sup")')
    blue_sup_name = blue_sup_name['player'].iloc[0]
    # red players
    red_frame = df.query(f'(gameid == "{gameid}") and (side == "Red")')
    red_top_name = red_frame.query(f'(gameid == "{gameid}") and (side == "Red") and (position == "top")')
    red_top_name = red_top_name['player'].iloc[0]
    red_jng_name = red_frame.query(f'(gameid == "{gameid}") and (side == "Red") and (position == "jng")')
    red_jng_name = red_jng_name['player'].iloc[0]
    red_mid_name = red_frame.query(f'(gameid == "{gameid}") and (side == "Red") and (position == "mid")')
    red_mid_name = red_mid_name['player'].iloc[0]
    red_bot_name = red_frame.query(f'(gameid == "{gameid}") and (side == "Red") and (position == "bot")')
    red_bot_name = red_bot_name['player'].iloc[0]
    red_sup_name = red_frame.query(f'(gameid == "{gameid}") and (side == "Red") and (position == "sup")')
    red_sup_name = red_sup_name['player'].iloc[0]
    # date, teams, result
    date = blue_frame['date'].iloc[0]
    blue_team_result = blue_frame['result'].iloc[0]
    blue_team = blue_frame['team'].iloc[0]
    red_team = red_frame['team'].iloc[0]
    # create pandas series for df
    match = pd.Series([gameid, date, \
                       blue_team, \
                       blue_top_name, \
                       blue_jng_name, \
                       blue_mid_name, \
                       blue_bot_name, \
                       blue_sup_name,\
                       blue_team_result, \
                       red_team, \
                       red_top_name, \
                       red_jng_name, \
                       red_mid_name, \
                       red_bot_name, \
                       red_sup_name])
    return match


lcs_rating[['gameid', 'date', \
           'blue_team', \
           'blue_top_name', \
           'blue_jng_name', \
           'blue_mid_name', \
           'blue_bot_name', \
           'blue_sup_name',\
           'blue_team_result', \
           'red_team', \
           'red_top_name', \
           'red_jng_name', \
           'red_mid_name', \
           'red_bot_name', \
           'red_sup_name']] = input_data.apply(lambda row: setup_match(input_data, row['gameid'], row['position']),axis=1)


In [15]:
lcs_rating.shape


(7590, 15)

In [16]:
lcs_rating = lcs_rating.reset_index(drop=True)
lcs_rating = lcs_rating.drop_duplicates(subset=['gameid'])
lcs_rating = lcs_rating.sort_values(by=['date'])

In [17]:
lcs_rating.head(20)

,gameid,date,blue_team,blue_top_name,blue_jng_name,blue_mid_name,blue_bot_name,blue_sup_name,blue_team_result,red_team,red_top_name,red_jng_name,red_mid_name,red_bot_name,red_sup_name
0,TRLH1/1002440062,2018-01-20 20:50:09,Team Liquid,Impact,Xmithie,Pobelter,Doublelift,Olleh,1,TSM,Hauntzer,MikeYeung,Bjergsen,Zven,Mithy
10,TRLH1/1002440076,2018-01-20 22:38:20,100 Thieves,Ssumday,Meteos,Ryu,Cody Sun,aphromoo,1,OpTic Gaming,zig,Akaadian,PowerOfEvil,Arrow,LemonNation
20,TRLH1/1002440084,2018-01-21 00:14:20,Clutch Gaming,Solo,Lira,FEBIVEN,Apollo,Hakuho,1,Golden Guardians,Lourlo,Contractz,Hai,Deftly,Matt
30,TRLH1/1002440095,2018-01-21 01:20:39,Echo Fox,Huni,Dardoch,Fenix,Altec,Adrian,1,FlyQuest,Flame,AnDa,Keane,WildTurtle,Stunt
40,TRLH1/1002440106,2018-01-21 02:16:56,Counter Logic Gaming,ZionSpartan,Reignover,huhi,Stixxay,Biofrost,0,Cloud9,Licorice,Svenskeren,Jensen,Sneaky,Smoothie
50,TRLH1/1002440127,2018-01-21 21:49:32,OpTic Gaming,zig,Akaadian,PowerOfEvil,Arrow,LemonNation,0,Team Liquid,Impact,Xmithie,Pobelter,Doublelift,Olleh
60,TRLH1/1002440132,2018-01-21 22:46:16,FlyQuest,Flame,AnDa,Keane,WildTurtle,Stunt,1,TSM,Hauntzer,MikeYeung,Bjergsen,Zven,Mithy
70,TRLH1/1002440143,2018-01-22 00:00:28,100 Thieves,Ssumday,Meteos,Ryu,Cody Sun,aphromoo,1,Counter Logic Gaming,ZionSpartan,Reignover,huhi,Stixxay,Biofrost
80,TRLH1/1002440150,2018-01-22 01:00:51,Golden Guardians,Lourlo,Contractz,Hai,Deftly,Matt,0,Cloud9,Licorice,Svenskeren,Jensen,Sneaky,Smoothie
90,TRLH1/1002440161,2018-01-22 02:02:56,Echo Fox,Huni,Dardoch,Fenix,Altec,Adrian,1,Clutch Gaming,Solo,Lira,FEBIVEN,Apollo,Hakuho


In [18]:
analyzed_gameids = {}

def win_probability(team1, team2, trueskill_global_env):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (trueskill_global_env.beta ** 2 ) + sum_sigma)
    return trueskill_global_env.cdf(delta_mu / denom)

def update_trueskill(rating_dict, gameid_dict, gameid, blue_team_result, \
                        blue_top_name, \
                        blue_jng_name, \
                        blue_mid_name, \
                        blue_bot_name, \
                        blue_sup_name,\
                        red_top_name, \
                        red_jng_name, \
                        red_mid_name, \
                        red_bot_name, \
                        red_sup_name):
    rating_groups = [(rating_dict[blue_top_name], \
                      rating_dict[blue_jng_name], \
                      rating_dict[blue_mid_name], \
                      rating_dict[blue_bot_name], \
                      rating_dict[blue_sup_name]), \
                     (rating_dict[red_top_name], \
                      rating_dict[red_jng_name], \
                      rating_dict[red_mid_name], \
                      rating_dict[red_bot_name], \
                      rating_dict[red_sup_name])]
    blue_mu = rating_groups[0][0].mu
    blue_sigma = rating_groups[0][0].sigma
    red_mu = rating_groups[1][0].mu
    red_sigma = rating_groups[1][0].sigma
    blue_team_win_prob = win_probability(rating_groups[0], rating_groups[1], ts)
    # Get Mu by position
    blue_top_mu = rating_dict[blue_top_name].mu
    blue_jng_mu = rating_dict[blue_jng_name].mu
    blue_mid_mu = rating_dict[blue_mid_name].mu
    blue_bot_mu = rating_dict[blue_bot_name].mu
    blue_sup_mu = rating_dict[blue_sup_name].mu
    red_top_mu = rating_dict[red_top_name].mu
    red_jng_mu = rating_dict[red_jng_name].mu
    red_mid_mu = rating_dict[red_mid_name].mu
    red_bot_mu = rating_dict[red_bot_name].mu
    red_sup_mu = rating_dict[red_sup_name].mu
    # Get Sigma by position
    blue_top_sigma = rating_dict[blue_top_name].sigma
    blue_jng_sigma = rating_dict[blue_jng_name].sigma
    blue_mid_sigma = rating_dict[blue_mid_name].sigma
    blue_bot_sigma = rating_dict[blue_bot_name].sigma
    blue_sup_sigma = rating_dict[blue_sup_name].sigma
    red_top_sigma = rating_dict[red_top_name].sigma
    red_jng_sigma = rating_dict[red_jng_name].sigma
    red_mid_sigma = rating_dict[red_mid_name].sigma
    red_bot_sigma = rating_dict[red_bot_name].sigma
    red_sup_sigma = rating_dict[red_sup_name].sigma
    # Update ratings
    if blue_team_result == 1:
        rated_rating_groups = ts.rate(rating_groups, ranks=[0, 1]) # for ranks 0 is winner 
    else:
        rated_rating_groups = ts.rate(rating_groups, ranks=[1, 0])
    # Return values for new columns
    ts_update = pd.Series([blue_team_win_prob, blue_mu, blue_sigma, red_mu, red_sigma, \
           blue_top_mu, blue_top_sigma, \
           blue_jng_mu, blue_jng_sigma, \
           blue_mid_mu, blue_mid_sigma, \
           blue_bot_mu, blue_bot_sigma, \
           blue_sup_mu, blue_sup_sigma, \
           red_top_mu, red_top_sigma, \
           red_jng_mu, red_jng_sigma, \
           red_mid_mu, red_mid_sigma, \
           red_bot_mu, red_bot_sigma, \
           red_sup_mu, red_sup_sigma])
    print(ts_update)
    # update the rating dictionary
    rating_dict[blue_top_name] = rated_rating_groups[0][0]
    rating_dict[blue_jng_name] = rated_rating_groups[0][1]
    rating_dict[blue_mid_name] = rated_rating_groups[0][2]
    rating_dict[blue_bot_name] = rated_rating_groups[0][3]
    rating_dict[blue_sup_name] = rated_rating_groups[0][4]
    rating_dict[red_top_name] = rated_rating_groups[1][0]
    rating_dict[red_jng_name] = rated_rating_groups[1][1]
    rating_dict[red_mid_name] = rated_rating_groups[1][2]
    rating_dict[red_bot_name] = rated_rating_groups[1][3]
    rating_dict[red_sup_name] = rated_rating_groups[1][4]
    # determine if use old data because of repeat
    if gameid in gameid_dict:
        return gameid_dict[gameid]
    else:
        gameid_dict[gameid] = ts_update
        return ts_update

lcs_rating[['blue_team_win_prob', 'blue_mu', 'blue_sigma', 'red_mu', 'red_sigma',\
           'blue_top_mu', 'blue_top_sigma', \
           'blue_jng_mu', 'blue_jng_sigma', \
           'blue_mid_mu', 'blue_mid_sigma', \
           'blue_bot_mu', 'blue_bot_sigma', \
           'blue_sup_mu', 'blue_sup_sigma', \
           'red_top_mu', 'red_top_sigma', \
           'red_jng_mu', 'red_jng_sigma', \
           'red_mid_mu', 'red_mid_sigma', \
           'red_bot_mu', 'red_bot_sigma', \
           'red_sup_mu', 'red_sup_sigma']] = lcs_rating.apply(lambda row: update_trueskill(player_ratings_dict, analyzed_gameids, row['gameid'], row['blue_team_result'], \
                        row['blue_top_name'], \
                        row['blue_jng_name'], \
                        row['blue_mid_name'], \
                        row['blue_bot_name'], \
                        row['blue_sup_name'],\
                        row['red_top_name'], \
                        row['red_jng_name'], \
                        row['red_mid_name'], \
                        row['red_bot_name'], \
                        row['red_sup_name']), axis=1)

0      0.500000
1     25.000000
2      8.333333
3     25.000000
4      8.333333
5     25.000000
6      8.333333
7     25.000000
8      8.333333
9     25.000000
10     8.333333
11    25.000000
12     8.333333
13    25.000000
14     8.333333
15    25.000000
16     8.333333
17    25.000000
18     8.333333
19    25.000000
20     8.333333
21    25.000000
22     8.333333
23    25.000000
24     8.333333
dtype: float64
0      0.742714
1     26.880745
2      8.118755
3     23.119255
4      8.118755
5     26.880745
6      8.118755
7     26.880745
8      8.118755
9     26.880745
10     8.118755
11    26.880745
12     8.118755
13    26.880745
14     8.118755
15    23.119255
16     8.118755
17    23.119255
18     8.118755
19    23.119255
20     8.118755
21    23.119255
22     8.118755
23    23.119255
24     8.118755
dtype: float64
0      0.500000
1     25.000000
2      8.333333
3     25.000000
4      8.333333
5     25.000000
6      8.333333
7     25.000000
8      8.333333
9     25.000000
10     8.3

dtype: float64
0      0.445731
1     28.451368
2      3.628099
3     26.875382
4      3.622790
5     28.451368
6      3.628099
7     28.451368
8      3.628099
9     26.307604
10     4.586441
11    26.234986
12     5.062075
13    22.387942
14     4.013318
15    26.875382
16     3.622790
17    26.824923
18     4.316536
19    26.875382
20     3.622790
21    26.875382
22     3.622790
23    26.875382
24     3.622790
dtype: float64
0      0.716583
1     27.538323
2      3.657259
3     23.608266
4      4.232445
5     27.538323
6      3.657259
7     24.340580
8      4.394820
9     27.538323
10     3.657259
11    27.538323
12     3.657259
13    27.538323
14     3.657259
15    23.608266
16     4.232445
17    27.775830
18     4.944270
19    23.212483
20     4.709438
21    23.608266
22     4.232445
23    25.501075
24     5.091795
dtype: float64
0      0.793798
1     27.874015
2      3.623981
3     23.158739
4      4.180140
5     27.874015
6      3.623981
7     24.825247
8      4.336106
9     27.87

dtype: float64
0      0.317324
1     24.540662
2      2.881742
3     31.814661
4      6.064243
5     24.540662
6      2.881742
7     24.193663
8      3.200123
9     27.346073
10     4.040847
11    26.929736
12     2.953005
13    30.822491
14     3.184697
15    31.814661
16     6.064243
17    35.062386
18     4.310928
19    25.546358
20     3.268715
21    25.344452
22     3.295421
23    24.496175
24     3.250158
dtype: float64
0      0.312890
1     24.342161
2      3.532234
3     30.081284
4      6.676923
5     24.342161
6      3.532234
7     24.727710
8      3.069303
9     29.074442
10     2.611401
11    21.762496
12     2.862256
13    23.628483
14     7.984635
15    30.081284
16     6.676923
17    28.804254
18     6.478117
19    28.557533
20     3.133603
21    19.639750
22     4.447237
23    26.184808
24     2.801227
dtype: float64
0      0.855610
1     30.381786
2      2.705499
3     27.848108
4      2.970653
5     30.381786
6      2.705499
7     30.381786
8      2.705499
9     24.83

0      0.163733
1     24.499531
2      3.058814
3     29.828220
4      2.441722
5     24.499531
6      3.058814
7     24.834151
8      2.750080
9     26.174379
10     2.615766
11    21.850468
12     2.602035
13    30.879867
14     6.911333
15    29.828220
16     2.441722
17    30.682163
18     3.364938
19    27.676815
20     3.164492
21    31.385132
22     7.103290
23    26.619693
24     2.546756
dtype: float64
0      0.518263
1     28.243857
2      2.838621
3     29.181491
4      3.455096
5     28.243857
6      2.838621
7     27.125653
8      3.317030
9     27.818178
10     2.239821
11    21.565388
12     7.071313
13    24.216140
14     2.613423
15    29.181491
16     3.455096
17    33.450826
18     3.031226
19    19.704699
20     3.421890
21    24.272003
22     2.623874
23    21.566825
24     2.429776
dtype: float64
0      0.395967
1     24.842020
2      5.653039
3     28.651385
4      3.412754
5     24.842020
6      5.653039
7     26.152726
8      2.608904
9     20.498566
10     6.8

dtype: float64
0      0.836981
1     28.568591
2      2.217018
3     30.397438
4      3.303540
5     28.568591
6      2.217018
7     34.409886
8      3.230990
9     33.641180
10     2.519818
11    30.379232
12     2.056421
13    28.049827
14     2.537772
15    30.397438
16     3.303540
17    27.589429
18     2.232820
19    27.849442
20     3.480745
21    20.581402
22     2.687176
23    33.223809
24     2.209571
dtype: float64
0      0.396861
1     23.704063
2      2.678581
3     26.267519
4      2.894831
5     23.704063
6      2.678581
7     26.157367
8      2.530703
9     25.655874
10     2.705115
11    22.600705
12     4.086251
13    25.281810
14     2.301212
15    26.267519
16     2.894831
17    31.141013
18     2.639389
19    27.643402
20     2.288305
21    22.438263
22     2.390416
23    20.025421
24     2.227160
dtype: float64
0      0.495301
1     30.192589
2      3.276942
3     29.874192
4      2.171070
5     30.192589
6      3.276942
7     27.495778
8      2.225848
9     27.62

dtype: float64
0      0.351049
1     25.953492
2      2.042209
3     30.804130
4      2.011369
5     25.953492
6      2.042209
7     27.197099
8      1.963515
9     28.379998
10     1.921129
11    25.004082
12     1.964013
13    38.098115
14     2.468811
15    30.804130
16     2.011369
17    27.307191
18     3.354458
19    25.287140
20     2.091899
21    38.011875
22     3.747071
23    29.076702
24     2.294919
dtype: float64
0      0.383078
1     30.202039
2      2.024121
3     30.524508
4      2.000667
5     30.202039
6      2.024121
7     27.528184
8      3.112936
9     29.258435
10     1.804935
11    30.526940
12     1.944240
13    25.706391
14     2.037848
15    30.524508
16     2.000667
17    26.530308
18     3.297623
19    24.984718
20     2.079577
21    37.042612
22     3.667175
23    28.712829
24     2.277967
dtype: float64
0      0.512978
1     30.264736
2      1.990546
3     30.467927
4      2.013557
5     30.264736
6      1.990546
7     25.825342
8      3.245399
9     24.70

In [19]:
lcs_rating.shape

(759, 40)

In [20]:
lcs_rating = lcs_rating.reset_index(drop=True)
lcs_rating.head(20)

,gameid,date,blue_team,blue_top_name,blue_jng_name,blue_mid_name,blue_bot_name,blue_sup_name,blue_team_result,red_team,red_top_name,red_jng_name,red_mid_name,red_bot_name,red_sup_name,blue_team_win_prob,blue_mu,blue_sigma,red_mu,red_sigma,blue_top_mu,blue_top_sigma,blue_jng_mu,blue_jng_sigma,blue_mid_mu,blue_mid_sigma,blue_bot_mu,blue_bot_sigma,blue_sup_mu,blue_sup_sigma,red_top_mu,red_top_sigma,red_jng_mu,red_jng_sigma,red_mid_mu,red_mid_sigma,red_bot_mu,red_bot_sigma,red_sup_mu,red_sup_sigma
0,TRLH1/1002440062,2018-01-20 20:50:09,Team Liquid,Impact,Xmithie,Pobelter,Doublelift,Olleh,1,TSM,Hauntzer,MikeYeung,Bjergsen,Zven,Mithy,0.500000,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333
1,TRLH1/1002440076,2018-01-20 22:38:20,100 Thieves,Ssumday,Meteos,Ryu,Cody Sun,aphromoo,1,OpTic Gaming,zig,Akaadian,PowerOfEvil,Arrow,LemonNation,0.500000,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333
2,TRLH1/1002440084,2018-01-21 00:14:20,Clutch Gaming,Solo,Lira,FEBIVEN,Apollo,Hakuho,1,Golden Guardians,Lourlo,Contractz,Hai,Deftly,Matt,0.500000,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333
3,TRLH1/1002440095,2018-01-21 01:20:39,Echo Fox,Huni,Dardoch,Fenix,Altec,Adrian,1,FlyQuest,Flame,AnDa,Keane,WildTurtle,Stunt,0.500000,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333
4,TRLH1/1002440106,2018-01-21 02:16:56,Counter Logic Gaming,ZionSpartan,Reignover,huhi,Stixxay,Biofrost,0,Cloud9,Licorice,Svenskeren,Jensen,Sneaky,Smoothie,0.500000,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333
5,TRLH1/1002440127,2018-01-21 21:49:32,OpTic Gaming,zig,Akaadian,PowerOfEvil,Arrow,LemonNation,0,Team Liquid,Impact,Xmithie,Pobelter,Doublelift,Olleh,0.203328,23.119255,8.118755,27.872979,7.966146,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,27.872979,7.966146,27.872979,7.966146,27.872979,7.966146,27.872979,7.966146,27.872979,7.966146
6,TRLH1/1002440132,2018-01-21 22:46:16,FlyQuest,Flame,AnDa,Keane,WildTurtle,Stunt,1,TSM,Hauntzer,MikeYeung,Bjergsen,Zven,Mithy,0.568753,23.119255,8.118755,22.127021,7.966146,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,22.127021,7.966146,22.127021,7.966146,22.127021,7.966146,22.127021,7.966146,22.127021,7.966146
7,TRLH1/1002440143,2018-01-22 00:00:28,100 Thieves,Ssumday,Meteos,Ryu,Cody Sun,aphromoo,1,Counter Logic Gaming,ZionSpartan,Reignover,huhi,Stixxay,Biofrost,0.742714,26.880745,8.118755,23.119255,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755
8,TRLH1/1002440150,2018-01-22 01:00:51,Golden Guardians,Lourlo,Contractz,Hai,Deftly,Matt,0,Cloud9,Licorice,Svenskeren,Jensen,Sneaky,Smoothie,0.257286,23.119255,8.118755,26.880745,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755
9,TRLH1/1002440161,2018-01-22 02:02:56,Echo Fox,Huni,Dardoch,Fenix,Altec,Adrian,1,Clutch Gamin

In [21]:
#print(analyzed_gameids)

In [22]:
lcs_rating.to_csv('./lcs_player_trueskill_2018-2021.csv', index=False)